In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ 成功讀取 OPENAI_API_KEY")
else:
    print("❌ 沒有讀取到 OPENAI_API_KEY，請檢查 .env 檔")

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=1)
response = llm.invoke("hello")
print("✅ LLM 測試回應：", response.content)


✅ 成功讀取 OPENAI_API_KEY
✅ LLM 測試回應： Hello! How can I assist you today?


# Json版

In [ ]:
# semantic_retrieval_gpt4o.py
import os
import pandas as pd
from langchain.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ 成功讀取 OPENAI_API_KEY")
else:
    print("❌ 沒有讀取到 OPENAI_API_KEY，請檢查 .env 檔")

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=1)
response = llm.invoke("hello")
print("✅ LLM 測試回應：", response.content)

# ✅ 讀取資料
# ✅ 載入資料
text_units = pd.read_json("50PDF0409_text_units.json", lines=True)
embeddings = pd.read_json("50PDF0409_embeddings.text_unit.text.json", lines=True)

id2text = dict(zip(text_units["id"], text_units["text"]))
id2embedding = dict(zip(embeddings["id"], embeddings["embedding"]))
documents = [
    Document(page_content=id2text[_id], metadata={"id": _id})
    for _id in id2text if _id in id2embedding
]

# ✅ 建立向量庫 (Chroma)
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# ✅ 初始化 GPT-4o
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.8)

# ✅ 提問
query = "哪些高熵合金常被用於催化反應？"
docs = retriever.invoke(query)

# ✅ 顯示檢索結果
print("\n🔍 前 5 筆檢索段落：\n")
for i, doc in enumerate(docs):
    snippet = doc.page_content[:100].replace("\n", " ")
    print(f"{i+1}. (id: {doc.metadata['id']}) {snippet}...\n")

# ✅ 合成 Prompt 並詢問 GPT-4o
context = "\n\n".join(doc.page_content for doc in docs)
prompt_template = ChatPromptTemplate.from_template(
    """
你是一位專精於材料科學的助理，請根據以下段落回答使用者的問題。
=== 檢索內容 ===
{context}

=== 使用者問題 ===
{question}
"""
)
prompt = prompt_template.format_messages(context=context, question=query)
response = llm.invoke(prompt)

print("🧠 GPT-4o 回答：\n" + "-" * 50 + f"\n{response.content}\n")

In [ ]:
# semantic_rerank_gpt4o.py
import os
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from sentence_transformers import CrossEncoder

from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ 成功讀取 OPENAI_API_KEY")
else:
    print("❌ 沒有讀取到 OPENAI_API_KEY，請檢查 .env 檔")

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=1)
response = llm.invoke("hello")
print("✅ LLM 測試回應：", response.content)
# ✅ 載入文本與嵌入資料
text_units = pd.read_json("50PDF0409_text_units.json", lines=True)
embeddings = pd.read_json("50PDF0409_embeddings.text_unit.text.json", lines=True)

id2text = dict(zip(text_units["id"], text_units["text"]))
id2embedding = dict(zip(embeddings["id"], embeddings["embedding"]))

documents = [
    Document(page_content=id2text[_id], metadata={"id": _id})
    for _id in id2text if _id in id2embedding
]

# ✅ Chroma 向量資料庫（暫存）
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# ✅ 查詢
query = "哪些高熵合金常被應用於催化？"
initial_docs = retriever.invoke(query)

# ✅ CrossEncoder 精細排序
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
rerank_inputs = [(query, doc.page_content) for doc in initial_docs]
scores = reranker.predict(rerank_inputs)

# ✅ 排序並取前 5
scored_docs = list(zip(scores, initial_docs))
scored_docs.sort(key=lambda x: x[0], reverse=True)
top_docs = [doc for _, doc in scored_docs[:5]]

# ✅ 顯示 top 5
print("\n🔍 向量檢索 + CrossEncoder 重排序前 5：\n")
for i, doc in enumerate(top_docs):
    print(f"{i+1}. (id: {doc.metadata['id']}) {doc.page_content[:100]}...\n")

# ✅ GPT-4o 回答
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
context = "\n\n".join(doc.page_content for doc in top_docs)

prompt_template = ChatPromptTemplate.from_template(
    """
你是一位材料科學研究助理，請根據以下段落回答使用者的問題。
=== 檢索內容 ===
{context}

=== 使用者問題 ===
{question}
"""
)
prompt = prompt_template.format_messages(context=context, question=query)
response = llm.invoke(prompt)

print("🧠 GPT-4o 回答：\n" + "-" * 50 + f"\n{response.content}\n")


c:\Users\chen\anaconda3\envs\langchainragas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\chen\AppData\Local\Temp\ipykernel_38316\330972986.py:28: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")



🔍 向量檢索 + CrossEncoder 重排序前 5：

1. (id: bdaf8a8137198e7e00fab055c91b2042f1e715aa02c9eb199f5058cba9b47fcc25408af3cb19046da99f6ceb06ed3f1813e92f23ba98ae2288d80058dfd1525c)  the product, but also shape, size, and distri-
bution of the pores in the proposed structure. This ...

2. (id: 38424cc2d18388a3cd91e0cd84d2ef1b0e32006753c6dbc45b1e24f0e6448f2106fa190335913a0174069d738659b1c04a31f766bf50f2cb75eb6fda037f4b89) nic Technology Co., Ltd. after single ‐sided polishing
treatment. 2,3,6,7,10,11 ‐hexaaminotriphenyle...

3. (id: 4a20a58e3757134db8b62ef79183ee52f9202aaf2b227ad2551a518de9c57d98c87af5ed529083cce83bbd7175a743ece27267f85bb0f47ffff9d3e844b27ed8) tes [ 6]. In the MABs with alkaline electrolytes, KOH, NaOH,
and LiOH are the most commonly used. KO...

4. (id: a80a08afa3d9c10efea9935f693b628bef96c4a13d246a41a785a74c306b5211b7ee6676d05f1467c3349631b0294ea45ebd100dc51414b0344a1f91341010ea) materials
Article
Corrosion Resistance of the CpTi G2 Cellular Lattice with
TPMS Architecture for Ga.

C:\Users\chen\AppData\Local\Temp\ipykernel_38316\330972986.py:57: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


🧠 GPT-4o 回答：
--------------------------------------------------
在檢索內容中並未直接提到高熵合金及其在催化中的應用。因此，根據提供的資料，我無法回答這個問題。高熵合金是一類由多種元素組成的合金，通常具有優異的機械性能和耐腐蝕性，並在催化、能源和航空航天等領域有潛在應用。如果需要更詳細的信息，建議查閱相關的科學文獻或資料庫。



In [7]:
vecs = embedding_model.embed_documents(["test"])
print(len(vecs[0]))


3072


# PDF版


In [26]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureChatOpenAI,AzureOpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.vectorstores import Chroma
# from qdrant_client import QdrantClient
# from langchain_qdrant import QdrantVectorStore
from langchain_core.runnables.history import RunnableWithMessageHistory
from typing import Dict
from langchain.schema.runnable import RunnableMap, RunnableLambda

In [15]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ 成功讀取 OPENAI_API_KEY")
else:
    print("❌ 沒有讀取到 OPENAI_API_KEY，請檢查 .env 檔")

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=1)
response = llm.invoke("hello")
print("✅ LLM 測試回應：", response.content)


✅ 成功讀取 OPENAI_API_KEY
✅ LLM 測試回應： Hello! How can I assist you today?


In [16]:
generator_llm = ChatOpenAI(
    model_name="gpt-4o",   # 或 "gpt-4", "gpt-3.5-turbo"
    temperature=0.3,       # 控制回答穩定性。推薦值：0~0.3（任務導向），0.7~1（創意任務）
    max_tokens=8192,       # 控制回應長度。推薦根據任務調整，如摘要可短，技術問答可設長
    # top_p=1,               # Nucleus sampling，與 temperature 二選一，預設不動
    frequency_penalty=1,   # 減少重複字詞（0~2）
    # presence_penalty=0,    # 鼓勵新話題（0~2）

)

WARNING! frequency_penalty is not default parameter.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.


In [ ]:
# ✅ 批次載入整個 PDF 資料夾
pdf_loader = DirectoryLoader(
    path="../PDF",                     # ← 根據你 notebook 所在位置調整
    glob="**/*.pdf",                  # 所有子目錄下的 PDF 都抓
    loader_cls=PyPDFLoader,           # 每一份 PDF 使用 PyPDFLoader 解析
    show_progress=True                # 顯示進度
)

documents = pdf_loader.load()         # 讀取所有 PDF 文件物件
print(f"總共載入 {len(documents)} 篇文件")  # 檢查是否讀到所有文件

# # 使用 PyPDFDirectoryLoader（每個 PDF 只給一個 document）
# pdf_loader = PyPDFDirectoryLoader("../PDF")
# documents = pdf_loader.load()

# print(f"總共載入 {len(documents)} 份 PDF 文件")  # 就會是真正的 PDF 檔數



# ✅ 分段：一份 PDF 切成多個 chunk（準備送嵌入）
splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=200
)
splits = splitter.split_documents(documents)
print(f"總共分割成 {len(splits)} 個 chunk")

100%|██████████| 61/61 [00:29<00:00,  2.07it/s]

總共載入 799 篇文件
總共分割成 8605 個 chunk


In [20]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    chunk_size=16  # 每次送 16 段去嵌入
)

# ✅ 建立向量庫
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    # persist_directory="./chroma_db"  # 可換成你想存的位置
)


C:\Users\chen\AppData\Local\Temp\ipykernel_38604\2721908531.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(


In [33]:
prompt = ChatPromptTemplate.from_messages([
("system", 
     """You are a research assistant specializing in materials science and engineering, with expertise in literature summarization, property analysis, and application interpretation.
Please help the user search for and organize information related to materials according to the following principles:

- Your responses must be based on reliable sources or literature, and all sources should be clearly cited.
- Do not fabricate data. If data is missing or unavailable, explicitly state “no data available” or “not mentioned in the literature.”
- If multiple entries meet the query criteria, present them in a comparison table.
- Use clear, objective, and academic language suitable for direct inclusion in technical reports.
- Please format each summary using the following structure:
    \n\n{context}"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

In [ ]:
# ✅ 建立 Retriever
# retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [35]:
def retrieve_and_concat(inputs: Dict[str, str]) -> str:
    query = inputs["question"]
    top_k = inputs.get("top_k", 5)

    # 🟡 使用動態設定的 k
    custom_retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})
    docs = custom_retriever.get_relevant_documents(query)

    context = ""
    for i, doc in enumerate(docs):
        meta = doc.metadata
        context += f"[{i+1}] 來源：{meta.get('source', '未知')}，第 {meta.get('page', '?')} 頁\n"
        context += doc.page_content + "\n\n"
    return context

retriever_chain = RunnableLambda(retrieve_and_concat)

# ✅ QA Chain 組合
qa_chain = (
    RunnableMap({
        "context": retriever_chain,
        "question": lambda x: x["question"],
        "history": lambda x: [] 
    })
    | prompt
    | generator_llm
    | StrOutputParser()
)

In [37]:
# ✅ 使用範例（含 top_k）
query_input = {
    "question": "請問有哪些高熵合金適合應用於氧還原反應（ORR）？",
    "top_k": 5
}

print("🧪 問題：", query_input["question"])

response = qa_chain.invoke(query_input)
print("🤖 回答：", response)

🧪 問題： 請問有哪些高熵合金適合應用於氧還原反應（ORR）？
🤖 回答： 以下是一些適合用於氧還原反應（ORR）的高熵合金，根據不同的文獻資料進行整理：

| 合金組成 | 特性 | 來源 |
|---------|------|------|
| CoCrFeNiPd | 具有優異的催化活性和穩定性，適合在酸性介質中使用。 | [1] Yu, 2022, Carbon Energy, 第42頁 |
| FeCoNiCuPt | 展示出高效的氧還原催化性能，特別是在鹼性介質中。 | [2] Löffler et al., Adv Energy Mater, 2018; 8(34):1802269 |
| PtRuCuOsIr (多孔結構) | 增強了甲醇氧化和氧還原活性的電催化性能。 | [3] Chen et al., Journal of Power Sources, 2015 |

這些高熵合金因其多元組分和獨特的微觀結構，使其在電催化應用中展現出優異的性能。

來源：
[1] Yu - High‐entropy alloy catalysts: From bulk to nano toward highly efficient carbon and nitrogen catalysis - Carbon Energy
[2] Löffler T , Meyer H , Savan A , et al. Discovery of a multinary noble metal-free oxygen reduction catalyst . Adv Energy Mater . 2018 ; 8(34):1802269.
[3] Chen X., Si C., Gao Y., Frenzel J., Sun J., Eggeler G. and Zhang Z. Multi-component nanoporous platinum–ruthenium–copper–osmium–iridium alloy with enhanced electrocatalytic activity towards methanol oxidation and oxygen reduction. Journal of Power Sources, 273:324-332.

In [39]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory



In [40]:
def get_session_history(session_id: str):
    return InMemoryChatMessageHistory()

In [43]:
qa_chain_with_memory = RunnableWithMessageHistory(
    qa_chain,                        # 你原本的 QA chain
    get_session_history,            # 記憶取得函式
    input_messages_key="question",  # 問題在哪個欄位
    # output_messages_key="answer",   # 回答會存在這裡
    history_messages_key="history"  # prompt 需要的 history
)


In [44]:
config = {
    "configurable": {
        "session_id": "user_123"  # 每位使用者或對話一個唯一 ID
    }
}

# 第一句
qa_chain_with_memory.invoke(
    {"question": "請問有哪些高熵合金適合應用於ORR？", "top_k": 5},
    config=config
)

# 第二句（上下文會接著上面）
qa_chain_with_memory.invoke(
    {"question": "哪一組合金的表現最好？", "top_k": 3},
    config=config
)


'為了評估哪一組合金的表現最好，我們需要考慮多個因素，包括機械性能（如抗拉強度、延展性）、耐腐蝕性、熱穩定性等。以下是一些來自不同文獻的合金性能比較：\n\n| 合金類型 | 抗拉強度 (MPa) | 延展性 (%) | 耐腐蝕性 | 熱穩定性 |\n|----------|-----------------|------------|----------|----------|\n| CoCrFeMnNi 高熵合金 [2] | 700-900         | 40-50      | 良好     | 優良     |\n| FeCoNiCrTi0.2 高熵合金 [3]  | >1000           | 20-30      | 未提及   | 良好     |\n\n來源：\n[1] J.J. Kai, K. Lu, Y. Liu, C.T. Liu, Multicomponent intermetallic nanoparticles and superb mechanical behaviors of complex alloys, Science 362 (2018) 933 e937.\n[2] S.J. Sun et al., Modulating the prestrain history to optimize strength and ductility in CoCrFeMnNi high-entropy alloy, Scripta Mater., https://doi.org/10.1016/j.scriptamat.2019.01.012.\n[3] Y.Tong et al., Outstanding tensile properties of a precipitation-strengthened FeCoNiCrTi0.2 high-entropy alloy at room and cryogenic temperatures, Acta Mater.\n\n從上表可以看出，FeCoNiCrTi0.2 高熵合金在抗拉強度方面表現優異，但其延展性相對較低，而 CoCrFeMnNi 合金則在延展性和熱穩定性方面有更好的平衡。因此，哪一組合金“最好”取決於具體應用需求。如果需要高強度，可能選擇 FeCoNiCrTi0.2

In [45]:
# ✅ 指定對話內容與參數
query_input = {
    "question": "請問有哪些高熵合金適合應用於氧還原反應（ORR）？",
    "top_k": 5
}

# ✅ 指定對話 session ID（可換成使用者帳號、UUID 等）
session_id = "user_124"

# ✅ 呼叫包含記憶的 QA Chain
print("🧪 問題：", query_input["question"])

response = qa_chain_with_memory.invoke(
    query_input,
    config={"configurable": {"session_id": session_id}}  # ✅ 加上記憶管理
)

print("🤖 回答：", response)


🧪 問題： 請問有哪些高熵合金適合應用於氧還原反應（ORR）？
🤖 回答： 以下是一些適合應用於氧還原反應（ORR）的高熵合金，根據文獻中的資料整理如下：

| 合金組成 | 特性 | 來源 |
|---------|-----|------|
| 無貴金屬多元合金 | 發現了一種無貴金屬的多元氧還原催化劑，具有優異的催化活性。這種材料在降低成本和提高效率方面具有潛力。 | [1] Löffler T, Meyer H, Savan A, et al. Discovery of a multinary noble metal-free oxygen reduction catalyst. Adv Energy Mater. 2018; 8(34):1802269. |
| 多組分納米孔合金/(氧)氫氧化物 | 用於雙功能氧電催化和可充電鋅空氣電池，展示了良好的性能。這類材料在能量轉換裝置中有重要應用價值。 | [2] Fang G, Gao J, Lv J, et al. Multi-component nanoporous alloy/(oxy)hydroxide for bifunctional oxygen electrocatalysis and rechargeable Zn-air batteries. Appl Catal B Environ. 2020; 268:118431.|

這些高熵合金因其獨特的組成和結構，使其在ORR中展現出優異的性能，並且提供了降低貴重材料使用量的新途徑。

來源：
[1] Yu - High‐entropy alloy catalysts: From bulk to nano toward highly efficient carbon and nitrogen catalysis - Carbon Energy - Wiley Online Library.
[2] 同上文獻


In [46]:
query_input = {
    "question": "哪一組合金的表現最好？",
    "top_k": 3
}

print("🧪 問題：", query_input["question"])

response = qa_chain_with_memory.invoke(
    query_input,
    config={"configurable": {"session_id": session_id}}
)

print("🤖 回答：", response)


🧪 問題： 哪一組合金的表現最好？
🤖 回答： 要确定哪种合金表现最好，需要根据特定的性能指标进行比较，例如强度、延展性、耐腐蚀性等。以下是一些常见合金的性能比较，基于现有文献：

| 合金类型 | 强度 (MPa) | 延展性 (%) | 耐腐蚀性 | 参考文献 |
|----------|------------|------------|----------|----------|
| CoCrFeMnNi 高熵合金 | 600-700    | 50-60     | 良好     | [2] 来源：..\PDF\Influence-of-pre-deformation-on-the-precipitation-characte_2021_Journal-of-A.pdf，第 10 页 |
| FeCoNiCrTi0.2 高熵合金（在室温和低温下） | >1000      | >20       | 优秀     | [3] 来源：..\PDF\Cryogenic-strengthening-of-Fe27Co24Ni23Cr26-high-entr_2024_Materials-Science.pdf，第 11 页 |

从上述表格可以看出，FeCoNiCrTi0.2 高熵合金在强度和低温条件下的表现优于 CoCrFeMnNi 合金。然而，具体哪个合金“最好”还取决于应用场景。例如，如果需要更高的延展性而不是极限强度，那么 CoCrFeMnNi 合金可能更适用。

请注意，这只是一个简化的比较，实际选择应考虑更多因素以及具体应用需求。


In [47]:
def ask_with_memory(chain, question: str, session_id: str = "default", top_k: int = 5):
    query_input = {"question": question, "top_k": top_k}
    print("🧪 問題：", question)
    response = chain.invoke(query_input, config={"configurable": {"session_id": session_id}})
    print("🤖 回答：", response)
    return response


In [48]:
ask_with_memory(qa_chain_with_memory, "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？", session_id="chen01", top_k=100)
ask_with_memory(qa_chain_with_memory, "哪一組合金性能最好？", session_id="chen01", top_k=50)


🧪 問題： 請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？
🤖 回答： 在高熵合金（HEA）催化劑的應用中，常見的元素組合包括多種過渡金屬和貴金屬。以下是一些常見的元素組合：

1. **CoMoFeNiCu**：這種組合被用於氨分解反應，顯示出高活性和穩定性。[來源：Yu, 2022, Carbon Energy，第22頁]

2. **PtPdRhRuCe**：這些元素被用於氨氧化反應，在高溫下顯示出優異的轉化率和選擇性。[來源：Yu, 2022, Carbon Energy，第90頁]

3. **RuRhCoNiIr**：此五元合金在氨分解中表現出色，是一種有效且成本較低的替代品。[來源：Yu, 2022, Carbon Energy，第21頁]

4. **AuAgCuPdSi** 和 **IrCuPd** 等多元納米結構，被認為具有設計納米裝置的潛力。[來源：Yu, 2022, Carbon Energy，第8頁]

5. **AuWCoNiPtRuMoPdRhFeCrIrSnMn** 等15元素HEAs，用於穩定易氧化氧化物。[來源：Yu, 2022, Carbon Energy，第12頁]

6. **AgAuCuPdPt 和 CoCuGaNiZn** 被計算預測為具有高選擇性和活性的CO還原催化劑。[來源：Yu, 2022, Carbon Energy，第20-21頁]

這些組合展示了HEA催化劑在不同反應中的潛力，特別是在需要多重活性位點以促進複雜、多步驟反應時。
🧪 問題： 哪一組合金性能最好？
🤖 回答： 要比較不同合金的性能，我們需要考慮多種因素，包括強度、延展性、耐高溫性、耐腐蝕性等。以下是一些常見的高熵合金（HEA）及其性能特點：

1. **CoCrFeMnNi**：
   - 擁有良好的強度和韌性，適用於從室溫到低溫環境。
   - 在高溫下，單相HEA的強度相對較弱，需要考慮擴散驅動的相變和蠕變行為。[來源：5]

2. **AlCoCrFeNiTi**：
   - 具有優異的室溫機械性能。
   - 添加Al後會影響結構演化和拉伸性能。[來源：31]

3. **(Co30Fe45Ni25)0.8(Al40Si60)0.2**：
   - 結合了良好的機械、電氣和磁性質：高屈服強度1636 MPa，高電阻率68 μΩ·

'要比較不同合金的性能，我們需要考慮多種因素，包括強度、延展性、耐高溫性、耐腐蝕性等。以下是一些常見的高熵合金（HEA）及其性能特點：\n\n1. **CoCrFeMnNi**：\n   - 擁有良好的強度和韌性，適用於從室溫到低溫環境。\n   - 在高溫下，單相HEA的強度相對較弱，需要考慮擴散驅動的相變和蠕變行為。[來源：5]\n\n2. **AlCoCrFeNiTi**：\n   - 具有優異的室溫機械性能。\n   - 添加Al後會影響結構演化和拉伸性能。[來源：31]\n\n3. **(Co30Fe45Ni25)0.8(Al40Si60)0.2**：\n   - 結合了良好的機械、電氣和磁性質：高屈服強度1636 MPa，高電阻率68 μΩ·cm，相對較高的飽和感應1.24 Tesla，以及相對低矯頑力59.7 A/m。[來源：7]\n\n4. **Fe27Co24Ni23Cr26**：\n   - 適用於從低溫到高溫多功能應用，有著優異的強-延展協同效應和應變硬化能力。[來源：19]\n\n5. **Al0.3CoCrFeNi**：\n   - 可以通過調整微觀結構實現卓越的機械性能，如超細晶粒結構可增強其抗拉伸能力。[來源：48]\n\n這些合金各有其獨特優勢，具體哪一組合金最好取決於您所關心或需要解決問題中的具體需求。例如，如果您需要在極端低溫下使用材料，可能選擇具有良好韌性的材料更為重要；而如果是在腐蝕環境中使用，則耐腐蝕性的表現會更為關鍵。\n\n如需進一步比較或選擇某一特定用途上的最佳材料，可以提供更多詳細信息以便進行針對性的分析。'